# Labolatorium 1

## Rekurencyjne mnożenie macierzy metodą Binet'a

Pseudokod algorytmu:

## Rekurencyjne mnożenie macierzy metodą Strassena

Pseudokod algorytmu:

## Mnożenie macierzy metodą znalezioną przez AlphaTensor

Bibliografia: https://www.deepmind.com/blog/discovering-novel-algorithms-with-alphatensor